## Osadzenia

W naszym poprzednim przykładzie operowaliśmy na wysokowymiarowych wektorach typu bag-of-words o długości `vocab_size`, a także jawnie konwertowaliśmy niskowymiarowe wektory reprezentacji pozycyjnej na rzadkie reprezentacje typu one-hot. Taka reprezentacja one-hot nie jest efektywna pod względem pamięci, a dodatkowo każde słowo jest traktowane niezależnie od innych, tzn. zakodowane wektory one-hot nie wyrażają żadnego semantycznego podobieństwa między słowami.

W tej jednostce będziemy kontynuować eksplorację zbioru danych **News AG**. Na początek załadujmy dane i pobierzmy kilka definicji z poprzedniego notatnika.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Czym jest embedding?

Idea **embeddingu** polega na reprezentowaniu słów za pomocą gęstych wektorów o niższej wymiarowości, które w pewien sposób odzwierciedlają semantyczne znaczenie słowa. Później omówimy, jak tworzyć znaczące embeddingi słów, ale na razie potraktujmy embeddingi jako sposób na zmniejszenie wymiarowości wektora słowa.

Warstwa embeddingu przyjmuje słowo jako dane wejściowe i generuje wektor wyjściowy o określonym `embedding_size`. W pewnym sensie jest to bardzo podobne do warstwy `Linear`, ale zamiast przyjmować wektor zakodowany w formie one-hot, może przyjąć numer słowa jako dane wejściowe.

Używając warstwy embeddingu jako pierwszej warstwy w naszej sieci, możemy przejść od modelu bag-of-words do modelu **embedding bag**, w którym najpierw konwertujemy każde słowo w naszym tekście na odpowiadający mu embedding, a następnie obliczamy jakąś funkcję agregującą dla wszystkich tych embeddingów, taką jak `sum`, `average` lub `max`.

![Obraz przedstawiający klasyfikator embeddingu dla pięciu słów w sekwencji.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Nasza sieć neuronowa klasyfikatora rozpocznie się od warstwy embeddingu, następnie warstwy agregacji, a na końcu liniowego klasyfikatora:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Radzenie sobie z różną długością sekwencji zmiennych

W wyniku tej architektury, minibatche dla naszej sieci muszą być tworzone w określony sposób. W poprzedniej części, podczas korzystania z bag-of-words, wszystkie tensory BoW w minibatchu miały jednakowy rozmiar `vocab_size`, niezależnie od rzeczywistej długości sekwencji tekstowej. Gdy przechodzimy do osadzania słów (word embeddings), kończymy z różną liczbą słów w każdej próbce tekstowej, a podczas łączenia tych próbek w minibatche musimy zastosować pewne wypełnienie (padding).

Można to zrobić, stosując tę samą technikę, polegającą na dostarczeniu funkcji `collate_fn` do źródła danych:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Trenowanie klasyfikatora osadzeń

Teraz, gdy zdefiniowaliśmy odpowiedni ładowacz danych, możemy wytrenować model, korzystając z funkcji treningowej, którą zdefiniowaliśmy w poprzedniej jednostce:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Uwaga**: Trenujemy tutaj tylko na 25 tysiącach rekordów (mniej niż jedna pełna epoka) ze względu na czas, ale możesz kontynuować trening, napisać funkcję do trenowania przez kilka epok i eksperymentować z parametrem szybkości uczenia, aby osiągnąć wyższą dokładność. Powinieneś być w stanie osiągnąć dokładność na poziomie około 90%.


### Warstwa EmbeddingBag i reprezentacja sekwencji o zmiennej długości

W poprzedniej architekturze musieliśmy uzupełniać wszystkie sekwencje do tej samej długości, aby dopasować je do minibatcha. Nie jest to jednak najbardziej efektywny sposób reprezentowania sekwencji o zmiennej długości – innym podejściem byłoby użycie wektora **offset**, który przechowuje przesunięcia wszystkich sekwencji zapisanych w jednym dużym wektorze.

![Obraz przedstawiający reprezentację sekwencji za pomocą przesunięć](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Note**: Na powyższym obrazku pokazano sekwencję znaków, ale w naszym przykładzie pracujemy z sekwencjami słów. Jednak ogólna zasada reprezentowania sekwencji za pomocą wektora przesunięć pozostaje taka sama.

Aby pracować z reprezentacją przesunięć, używamy warstwy [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Jest ona podobna do `Embedding`, ale przyjmuje jako wejście wektor zawartości oraz wektor przesunięć, a także zawiera warstwę uśredniającą, która może być `mean`, `sum` lub `max`.

Oto zmodyfikowana sieć wykorzystująca `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Aby przygotować zestaw danych do treningu, musimy dostarczyć funkcję konwersji, która przygotuje wektor przesunięcia:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Należy zauważyć, że w przeciwieństwie do wszystkich poprzednich przykładów, nasza sieć teraz akceptuje dwa parametry: wektor danych i wektor przesunięcia, które mają różne rozmiary. Podobnie, nasz ładowacz danych dostarcza nam 3 wartości zamiast 2: zarówno wektory tekstowe, jak i przesunięcia są dostarczane jako cechy. Dlatego musimy nieco dostosować naszą funkcję treningową, aby to uwzględnić:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantyczne osadzenia: Word2Vec

W naszym poprzednim przykładzie warstwa osadzenia modelu nauczyła się mapować słowa na reprezentacje wektorowe, jednak ta reprezentacja nie miała dużego znaczenia semantycznego. Byłoby dobrze nauczyć się takiej reprezentacji wektorowej, w której podobne słowa lub synonimy odpowiadałyby wektorom bliskim sobie pod względem jakiejś odległości wektorowej (np. odległości euklidesowej).

Aby to osiągnąć, musimy wstępnie wytrenować nasz model osadzenia na dużym zbiorze tekstów w specyficzny sposób. Jednym z pierwszych sposobów trenowania semantycznych osadzeń jest metoda [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Opiera się ona na dwóch głównych architekturach, które są używane do tworzenia rozproszonej reprezentacji słów:

 - **Ciągły worek słów** (CBoW) — w tej architekturze trenujemy model, aby przewidywał słowo na podstawie otaczającego kontekstu. Mając n-gram $(W_{-2},W_{-1},W_0,W_1,W_2)$, celem modelu jest przewidzenie $W_0$ na podstawie $(W_{-2},W_{-1},W_1,W_2)$.
 - **Ciągły skip-gram** jest przeciwieństwem CBoW. Model wykorzystuje otaczające okno słów kontekstowych, aby przewidzieć bieżące słowo.

CBoW działa szybciej, podczas gdy skip-gram jest wolniejszy, ale lepiej radzi sobie z reprezentacją rzadkich słów.

![Obraz przedstawiający algorytmy CBoW i Skip-Gram do konwersji słów na wektory.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Aby eksperymentować z osadzeniem word2vec wstępnie wytrenowanym na zbiorze danych Google News, możemy użyć biblioteki **gensim**. Poniżej znajdujemy słowa najbardziej podobne do 'neural'.

> **Note:** Przy pierwszym tworzeniu wektorów słów ich pobieranie może zająć trochę czasu!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Możemy również obliczyć osadzenia wektorowe z wyrazu, które będą używane do trenowania modelu klasyfikacyjnego (pokazujemy tylko pierwsze 20 komponentów wektora dla przejrzystości):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Wspaniałą cechą osadzeń semantycznych jest to, że można manipulować kodowaniem wektorowym, aby zmieniać semantykę. Na przykład możemy poprosić o znalezienie słowa, którego reprezentacja wektorowa byłaby jak najbliższa słowom *król* i *kobieta*, a jak najdalsza od słowa *mężczyzna*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Zarówno CBoW, jak i Skip-Grams to osadzenia „predykcyjne”, ponieważ uwzględniają jedynie lokalne konteksty. Word2Vec nie wykorzystuje globalnego kontekstu.

**FastText** rozwija Word2Vec, ucząc się reprezentacji wektorowych dla każdego słowa oraz n-gramów znakowych zawartych w każdym słowie. Wartości tych reprezentacji są następnie uśredniane do jednego wektora na każdym kroku treningu. Chociaż dodaje to dużo dodatkowych obliczeń podczas wstępnego treningu, umożliwia osadzeniom słów kodowanie informacji o pod-słowach.

Inna metoda, **GloVe**, opiera się na idei macierzy współwystępowania i wykorzystuje metody neuronowe do dekompozycji macierzy współwystępowania na bardziej ekspresyjne i nieliniowe wektory słów.

Możesz eksperymentować, zmieniając osadzenia na FastText i GloVe, ponieważ gensim obsługuje kilka różnych modeli osadzeń słów.


## Korzystanie z wstępnie wytrenowanych osadzeń w PyTorch

Możemy zmodyfikować powyższy przykład, aby wstępnie wypełnić macierz w naszej warstwie osadzeń semantycznymi osadzeniami, takimi jak Word2Vec. Musimy jednak pamiętać, że słowniki wstępnie wytrenowanych osadzeń i naszego korpusu tekstowego prawdopodobnie nie będą się pokrywać, więc zainicjalizujemy wagi dla brakujących słów losowymi wartościami:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Teraz przejdźmy do trenowania naszego modelu. Zauważ, że czas potrzebny na trenowanie modelu jest znacznie dłuższy niż w poprzednim przykładzie, ze względu na większy rozmiar warstwy osadzania, a co za tym idzie, znacznie większą liczbę parametrów. Ponadto, z tego powodu możemy potrzebować trenować nasz model na większej liczbie przykładów, jeśli chcemy uniknąć przeuczenia.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

W naszym przypadku nie zauważamy dużego wzrostu dokładności, co prawdopodobnie wynika z dość różnych słownictw.  
Aby rozwiązać problem różnic w słownictwie, możemy zastosować jedno z następujących rozwiązań:  
* Ponowne wytrenowanie modelu word2vec na naszym słownictwie  
* Załadowanie naszego zbioru danych z użyciem słownictwa z wstępnie wytrenowanego modelu word2vec. Słownictwo używane do załadowania zbioru danych można określić podczas ładowania.  

Drugie podejście wydaje się łatwiejsze, zwłaszcza że framework `torchtext` w PyTorch zawiera wbudowaną obsługę osadzeń. Możemy na przykład utworzyć słownictwo oparte na GloVe w następujący sposób:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Załadowany słownik posiada następujące podstawowe operacje:
* Słownik `vocab.stoi` pozwala nam zamienić słowo na jego indeks w słowniku
* `vocab.itos` działa odwrotnie - zamienia liczbę na słowo
* `vocab.vectors` to tablica wektorów osadzeń, więc aby uzyskać osadzenie dla słowa `s`, musimy użyć `vocab.vectors[vocab.stoi[s]]`

Oto przykład manipulacji osadzeniami, aby zademonstrować równanie **kind-man+woman = queen** (musiałem nieco dostosować współczynnik, aby to zadziałało):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Aby wytrenować klasyfikator za pomocą tych osadzeń, najpierw musimy zakodować nasz zbiór danych za pomocą słownictwa GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Jak widzieliśmy powyżej, wszystkie osadzenia wektorów są przechowywane w macierzy `vocab.vectors`. Dzięki temu niezwykle łatwo jest załadować te wagi do wag warstwy osadzenia, używając prostego kopiowania:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Jednym z powodów, dla których nie obserwujemy znaczącego wzrostu dokładności, jest fakt, że niektóre słowa z naszego zbioru danych są nieobecne w wstępnie wytrenowanym słowniku GloVe, a zatem są zasadniczo ignorowane. Aby przezwyciężyć ten fakt, możemy wytrenować własne osadzenia na naszym zbiorze danych.


## Kontekstowe osadzenia

Jednym z kluczowych ograniczeń tradycyjnych wstępnie wytrenowanych reprezentacji osadzeń, takich jak Word2Vec, jest problem rozróżniania znaczeń słów. Chociaż wstępnie wytrenowane osadzenia mogą uchwycić część znaczenia słów w kontekście, każde możliwe znaczenie danego słowa jest kodowane w tym samym osadzeniu. Może to powodować problemy w modelach wykorzystujących te osadzenia, ponieważ wiele słów, takich jak słowo „grać”, ma różne znaczenia w zależności od kontekstu, w którym są używane.

Na przykład słowo „grać” w tych dwóch zdaniach ma zupełnie inne znaczenie:
- Poszedłem na **sztukę** do teatru.
- Jan chce **grać** ze swoimi przyjaciółmi.

Wstępnie wytrenowane osadzenia powyżej reprezentują oba te znaczenia słowa „grać” w tym samym osadzeniu. Aby przezwyciężyć to ograniczenie, musimy budować osadzenia oparte na **modelu językowym**, który jest wytrenowany na dużym korpusie tekstu i *wie*, jak słowa mogą być zestawiane w różnych kontekstach. Omówienie kontekstowych osadzeń wykracza poza zakres tego samouczka, ale wrócimy do tego tematu, gdy będziemy omawiać modele językowe w następnej jednostce.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za autorytatywne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
